In [1]:
# data processing
import datetime
import logging
import warnings
from dataclasses import dataclass
from pathlib import Path
from typing import Optional, Union

import datarobot as dr
import polars as pl

log = logging.getLogger(__name__)

dr_logger = logging.getLogger("datarobot.models.project")
for handler in dr_logger.handlers:
    if isinstance(handler, logging.StreamHandler):
        # log.info(f"Removing handler {handler}")
        dr_logger.removeHandler(handler)


warnings.filterwarnings(
    action="ignore", category=pl.exceptions.PolarsInefficientMapWarning
)
warnings.filterwarnings(action="ignore", category=dr.errors.DataRobotDeprecationWarning)


@dataclass(kw_only=True)
class Data:
    df: pl.DataFrame
    columns: Optional[list[str]] = None
    row_index: Optional[list[int]] = None
    date_partition_column: Optional[Union[str, list[datetime.datetime]]] = None
    partition_column: Optional[str] = None
    date_column: Optional[str] = None
    date_format: str = "%Y-%m-%d"
    target_column: str

    @staticmethod
    def load(load_path: Path, target_column: str, **kwargs) -> "Data":
        df = pl.read_csv(
            load_path,
        )
        return Data(df=df, target_column=target_column, **kwargs)

    @property
    def rendered_df(self) -> pl.DataFrame:
        df = self.df
        if self.columns is not None and len(self.columns):
            columns = self.columns
            if self.date_column is not None:
                columns = [self.date_column] + columns
            if self.target_column is not None:
                columns = [self.target_column] + columns
            if self.partition_column is not None:
                columns = [self.partition_column] + columns
            if self.date_partition_column is not None and isinstance(
                self.date_partition_column, str
            ):
                columns = [self.date_partition_column] + columns
            columns = set(columns)
            df = df.select(columns)
        if self.row_index is not None and len(self.row_index):
            df = df[self.row_index]
        if (
            self.date_column is not None
            and not df.dtypes[df.columns.index(self.date_column)] == pl.Datetime
        ):
            df = df.with_columns(
                pl.col(self.date_column).str.to_datetime(self.date_format)
            )
        return df

    def get_date_partitions(self):
        df = self.rendered_df

        if self.date_column is None:
            raise ValueError("date column is not set")
        start_date = df[self.date_column].min()
        end_date = df[self.date_column].max()
        if self.date_partition_column is None:
            return {"__all__": df}

        elif isinstance(self.date_partition_column, str):
            return {
                group: group_df
                for group, group_df in df.groupby([self.date_partition_column])
            }

        elif isinstance(self.date_partition_column, list) and isinstance(
            self.date_partition_column[0], datetime.datetime
        ):
            partition_dates = self.date_partition_column
            return {
                start_date: df.filter(
                    df[self.date_column].is_between(start_date, end_date)
                )
                for start_date, end_date in zip(
                    [start_date] + partition_dates, partition_dates + [end_date]
                )
            }

    def get_partitions(self):
        df = self.get_dataframe()
        if self.partition_column is not None:
            return {
                group: df_group
                for group, df_group in df.groupby([self.partition_column])
            }
        else:
            return {"__all__": df}


@dataclass(kw_only=True)
class TrainingData(Data): ...


class ExternalHoldoutData(Data): ...


class FeatureSelectionMethod: ...


def load(path: str) -> tuple[TrainingData, ExternalHoldoutData]: ...


def variable_downsampling(data: Data) -> Data: ...


def target_engineering(data: Data) -> Data: ...


def feature_selection(data: Data, method: FeatureSelectionMethod) -> Data: ...

In [2]:
pwd

'/home/lukas/code/testing/test_x_flow/recipe-xflow'

In [3]:
training_data = TrainingData.load(
    load_path=Path("include/x_flow/raw_data/DR_Demo_Bond_trading_RFQ_train.csv"),
    date_column="date",
    date_partition_column=[datetime.datetime(2018, 6, 3), datetime.datetime(2018, 6, 24)],
    date_format="%d/%m/%Y",
    target_column="Mid",
)
test_data = ExternalHoldoutData.load(
    load_path=Path("include/x_flow/raw_data/DR_Demo_Bond_trading_RFQ_test.csv"),
    date_column="date",
    date_format="%d/%m/%Y",
    target_column="Mid",
)

In [4]:
import datarobotx
from copy import deepcopy

In [5]:
from abc import ABC, abstractmethod

from utils.operator import Operator


class DataPreprocessor(ABC):
    def fit(self, df: Data) -> "DataPreprocessor":
        return self._fit(df)

    def transform(self, df: Data) -> Data:
        df = deepcopy(df)
        return self._transform(df)

    def fit_transform(self, df: Data) -> Data:
        df = deepcopy(df)
        return self._fit(df)._transform(df)

    @abstractmethod
    def _fit(self, df: Data) -> "DataPreprocessor": ...
    @abstractmethod
    def _transform(self, df: Data) -> Data: ...


class BinarizeData(DataPreprocessor):
    def __init__(
        self,
        threshold: float,
        operator: str,
        binarize_drop_regression_target=True,
        binarize_new_target_name="target_cat",
    ):
        self._threshold = threshold
        self._operator = operator
        self._binarize_drop_regression_target = binarize_drop_regression_target
        self._binarize_new_target_name = binarize_new_target_name

    def _fit(self, df: Data):
        return self

    def _transform(self, df: Data) -> Data:
        """helper function: binarize a target variable for classification"""
        categorical_data = df.rendered_df
        target_series = categorical_data[df.target_column]

        op_fun = Operator(operator=self._operator).apply_operation(self._threshold)

        categorical_data = categorical_data.with_columns(
            target_series.map_elements(op_fun, return_dtype=bool).alias(self._binarize_new_target_name)
        )
        if self._binarize_drop_regression_target:
            categorical_data.drop(df.target_column)

        df.df = categorical_data
        df.target_column = self._binarize_new_target_name

        return df

In [6]:
from utils.fire import FIRE as FireHelper
from datarobotx.idp.autopilot import get_or_create_autopilot_run
from datarobotx.idp.datasets import get_or_create_dataset_from_df
from datarobotx.idp.common.hashing import get_hash


In [7]:
class FIRE(DataPreprocessor):
    def __init__(
        self,
        endpoint: str,
        token: str,
        **kwargs
    ):
        self._endpoint = endpoint
        self._token = token
        self._fire_kwargs = kwargs

    def _fit(self, df: Data):
        dataset_id = get_or_create_dataset_from_df(
            endpoint=self._endpoint,
            token=self._token,
            data_frame=df.rendered_df.to_pandas(),
            name="fire_dataset",
        )
        fire_token = get_hash(**self._fire_kwargs)
        project_id = get_or_create_autopilot_run(
            endpoint=self._endpoint,
            token=self._token,
            dataset_id=dataset_id,
            name=f"fire_project [{fire_token}]",
            analyze_and_model_config={
                "target": df.target_column,
                "mode": "quick",
                "max_wait": 10000,
                "worker_count": -1,
            },
            advanced_options_config={
                "blend_best_models": False,
                "prepare_model_for_deployment": False,
                "min_secondary_validation_model_count": 0,
            },
        )

        fire = FireHelper.get(project_id=project_id)
        fire.main_feature_reduction(
            **self._fire_kwargs,
        )

        top_featurelist_name = fire.get_top_model().featurelist_name
        top_featurelist = fire.get_featurelist_by_name(top_featurelist_name)  # type: ignore
        features = top_featurelist.features  # type: ignore
        self._features = features
        return self

    def _transform(self, df: Data) -> Data:
        """helper function: select features from FIRE project"""
        df.columns = self._features
        return df

In [8]:
binarizer = BinarizeData(
    threshold=100, operator="<", binarize_drop_regression_target=True)

fire = FIRE(
    endpoint="https://app.datarobot.com/api/v2",
    token="NWQ1NDA3YTVmNTU1Y2QxZDQxNmQ1YTZjOklqemlsaHJucTZtdU1NVThMLWpCVldJeEhpYUhOcFVo",
    reduction_method="Rank Aggregation",
)

In [9]:
binarized_data = binarizer.fit_transform(training_data)
fire.fit(binarized_data)

[06/14/24 22:51:50] INFO     In progress: 0, queued: 0 (waited: 0s)                                 ]8;id=834823;file:///home/lukas/code/testing/test_x_flow/recipe-xflow/.venv/lib/python3.12/site-packages/datarobot/models/project.py\project.py]8;;\:]8;id=776802;file:///home/lukas/code/testing/test_x_flow/recipe-xflow/.venv/lib/python3.12/site-packages/datarobot/models/project.py#4041\4041]8;;\

[06/14/24 22:52:05] INFO     skipping autopilot for Reduced FL by Median Rank, top2                     ]8;id=142409;file:///home/lukas/code/testing/test_x_flow/recipe-xflow/src/utils/fire.py\fire.py]8;;\:]8;id=665178;file:///home/lukas/code/testing/test_x_flow/recipe-xflow/src/utils/fire.py#680\680]8;;\

                    INFO     skipping autopilot for Reduced FL by Median Rank, top2                     ]8;id=868037;file:///home/lukas/code/testing/test_x_flow/recipe-xflow/src/utils/fire.py\fire.py]8;;\:]8;id=84409;file:///home/lukas/code/testing/test_x_flow/recipe-xflow/src/utils/fire.py#680\680]8;;\

[06/14/24 22:52:06] INFO     Run 1 completed                                                            ]8;id=886874;file:///home/lukas/code/testing/test_x_flow/recipe-xflow/src/utils/fire.py\fire.py]8;;\:]8;id=187720;file:///home/lukas/code/testing/test_x_flow/recipe-xflow/src/utils/fire.py#315\315]8;;\

                    INFO     Run 1 completed                                                            ]8;id=693024;file:///home/lukas/code/testing/test_x_flow/recipe-xflow/src/utils/fire.py\fire.py]8;;\:]8;id=726648;file:///home/lukas/code/testing/test_x_flow/recipe-xflow/src/utils/fire.py#315\315]8;;\

...

                    INFO     Run 4 completed                                                            ]8;id=961675;file:///home/lukas/code/testing/test_x_flow/recipe-xflow/src/utils/fire.py\fire.py]8;;\:]8;id=879119;file:///home/lukas/code/testing/test_x_flow/recipe-xflow/src/utils/fire.py#315\315]8;;\

[06/14/24 22:53:02] INFO     new model is worse.                                                        ]8;id=277096;file:///home/lukas/code/testing/test_x_flow/recipe-xflow/src/utils/fire.py\fire.py]8;;\:]8;id=329996;file:///home/lukas/code/testing/test_x_flow/recipe-xflow/src/utils/fire.py#304\304]8;;\
                             Repeat again. Decay Ratio for Simple and DR reduced methods                           

                    INFO     new model is worse.                                                        ]8;id=750026;file:///home/lukas/code/testing/test_x_flow/recipe-xflow/src/utils/fire.py\fire.py]8;;\:]8;id=754303;file:///home/lukas/code/testing/test_x_flow/recipe-xflow/src/utils/fire.py#304\304]8;;\
                             Repeat again. Decay Ratio for Simple and DR reduced methods                           

                    INFO     Run 5 completed                                                            ]8;id=794325;file:///home/lukas/code/testing/test_x_flow/recipe-xflow/src/utils/fire.py\fire.py]8;;\:]8;id=748132;file:///home/lukas/code/testing/test_x_flow/recipe-xflow/src/utils/fire.py#315\315]8;;\

                    INFO     Run 5 completed                                                            ]8;id=198787;file:///home/lukas/code/testing/test_x_flow/recipe-xflow/src/utils/fire.py\fire.py]8;;\:]8;id=558659;file:///home/lukas/code/testing/test_x_flow/recipe-xflow/src/utils/fire.py#315\315]8;;\

In [10]:
training_data.rendered_df.head()

request_id,date,cusip,BidAsk,Mid,yield_to_maturity,years_to_maturity,years_since_issue,is_bench_02y,is_bench_05y,is_bench_10y,is_bench_30y,IssueName,AmountOut,issue_date,Coupon,Currency,maturity_date,counterparty_id,counterparty_type,name,counterparty_aum,ann_account_value_bln,tier,salesperson,salesperson_num_ac_covered,notional_EURm,notional_bps_total_issue,log_notional,resp_bidAsk_norm,response_revenue_EUR,num_brokers,trade_won
i64,datetime[μs],str,f64,f64,f64,f64,f64,bool,bool,bool,bool,str,f64,str,f64,str,str,str,str,str,f64,f64,str,str,i64,i64,f64,f64,f64,f64,i64,bool
1777,2018-01-04 00:00:00,"""D2R8H4AK""",0.0227,102.16795,-0.522589,3.263585,2.507923,false,false,false,false,"""Government of Germany 0.25% 16…",20713.79945,"""03/07/2015""",0.25,"""Euro""","""10/04/2021""","""LO057""","""LO""","""Fayetteville Investments""",81.673308,33.523547,"""C""","""Meghan""",41,29,14.000329,1.462398,2.719329,4028.957313,10,true
1778,2018-03-06 00:00:00,"""D20658Z9""",0.1442,128.7033,1.284039,29.005387,4.01651,false,false,false,true,"""Government of Germany 2.5% 15-…",30525.59919,"""28/02/2014""",2.5,"""Euro""","""08/03/2047""","""LO111""","""LO""","""Knoxville Pensions""",114.119618,30.439696,"""C""","""Meghan""",41,208,68.139531,2.318063,12.510871,167553.6102,8,false
1779,2018-09-05 00:00:00,"""D206585A""",0.028,100.987,-0.607714,2.661246,3.616775,false,false,false,false,"""Government of Germany 0.0% 17-…",21803.99942,"""23/01/2015""",0.0,"""Euro""","""04/05/2021""","""LO140""","""LO""","""Cincinnati Capital""",210.929638,46.848707,"""A""","""Kate""",9,135,61.915247,2.130334,2.954119,20139.9049,9,false
1780,2019-08-14 00:00:00,"""D2R8H4DM""",0.023,109.0565,-0.721392,9.566247,1.086949,false,false,true,false,"""Government of Germany 0.25% 15…",22894.19939,"""13/07/2018""",0.25,"""Euro""","""08/03/2029""","""LO087""","""LO""","""Unalaska Advisors""",117.150554,14.189058,"""C""","""William""",15,99,43.242394,1.995635,2.661497,14369.06668,11,false
1781,2018-04-25 00:00:00,"""D20659WR""",0.0242,106.5311,-0.492709,1.957604,7.685305,true,false,false,false,"""Government of Germany 2.25% 04…",17443.19954,"""18/08/2010""",2.25,"""Euro""","""09/04/2020""","""LO090""","""LO""","""Jackson Trust""",135.431575,50.701944,"""B""","""Louis""",3,228,130.709965,2.357935,2.299022,27923.75113,9,false


In [11]:
fire.transform(binarizer.transform(training_data)).rendered_df.head()

years_to_maturity,yield_to_maturity,Coupon,date,target_cat,years_since_issue
f64,f64,f64,datetime[μs],bool,f64
3.263585,-0.522589,0.25,2018-01-04 00:00:00,false,2.507923
29.005387,1.284039,2.5,2018-03-06 00:00:00,false,4.01651
2.661246,-0.607714,0.0,2018-09-05 00:00:00,false,3.616775
9.566247,-0.721392,0.25,2019-08-14 00:00:00,false,1.086949
1.957604,-0.492709,2.25,2018-04-25 00:00:00,false,7.685305
